## Snowflake (On-Demand) Feature Group 
This tutorial notebook will show how you can define an external feature group for a table in Snowflake.

In this notebook we assume that you already have snowflake account and did the getting started with snowflake tutorial.

In [1]:
import com.logicalclocks.hsfs._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
0,application_1614610677629_0001,spark,idle,Link,Link


SparkSession available as 'spark'.
import com.logicalclocks.hsfs._


In [2]:
val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@295412ba
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=67, name='project1_featurestore', projectId=119, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@4851643}


In [ ]:
val snowflakeConn = fs.getStorageConnector("sfconnector")
var sfOptions = snowflakeConn.getSparkOptionsInt
sfOptions.put("query", "select * from TELCO")

In [4]:
val df = spark.read.format("net.snowflake.spark.snowflake").options(sfOptions).load()
df.show(10)

df: org.apache.spark.sql.DataFrame = [CUSTOMER_ID: string, GENDER: string ... 19 more fields]
+-----------+------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
|CUSTOMER_ID|GENDER|SENIOR_CITIZEN|PARTNER|DEPENDENTS|TENURE|PHONE_SERVICE|  MULTIPLE_LINES|INTERNET_SERVICE|ONLINE_SECURITY|ONLINE_BACKUP|DEVICE_PROTECTION|TECH_SUPPORT|STREAMING_TV|STREAMING_MOVIES|      CONTRACT|PAPERLESS_BILLING|      PAYMENT_METHOD|MONTHLY_CHARGES|TOTAL_CHARGES|CHURN|
+-----------+------+--------------+-------+----------+------+-------------+----------------+----------------+---------------+-------------+-----------------+------------+------------+----------------+--------------+-----------------+--------------------+---------------+-------------+-----+
| 7590-VHVEG|Female|         fals

### External (On-Demand) Feature Group
An on-Demand Feature Group stores the metadata for features in the feature store. The actual feature data is read from the external source using a storage connector. 
The next cell shows how to create an on-demand feature group using the snowflake connector.

In [5]:
val telcoOnDmd = (fs.createOnDemandFeatureGroup()
                    .name("telco_snowflake_scala")
                    .version(1)
                    .query("select * from CUSTOMER_CHURN")
                    .description("On-demand feature group for telecom customer data")
                    .storageConnector(snowflakeConn)
                    .statisticsConfig(new StatisticsConfig(true, true, true))
                    .build())

telcoOnDmd: com.logicalclocks.hsfs.OnDemandFeatureGroup = com.logicalclocks.hsfs.OnDemandFeatureGroup@55afc64e


In [6]:
telcoOnDmd.save()

In [14]:
import java.util.Arrays;
telcoOnDmd.select(Arrays.asList("customer_id", "internet_service", "phone_service", "total_charges", "churn")).show(5)

import java.util.Arrays
+-----------+----------------+-------------+-------------+-----+
|customer_id|internet_service|phone_service|total_charges|churn|
+-----------+----------------+-------------+-------------+-----+
| 7590-VHVEG|             DSL|           No|        29.85|   No|
| 5575-GNVDE|             DSL|          Yes|       1889.5|   No|
| 3668-QPYBK|             DSL|          Yes|       108.15|  Yes|
| 7795-CFOCW|             DSL|           No|      1840.75|   No|
| 9237-HQITU|     Fiber optic|          Yes|       151.65|  Yes|
+-----------+----------------+-------------+-------------+-----+
only showing top 5 rows

